In [1]:
import tensorflow as tf
import numpy as np

### Tensors

 Tensors are multidimensional arrays

In [28]:
x = tf.constant([[1, 2, 4],
                 [3, 5, 6]])

x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [29]:
print(f"Tensor x \n {x}")
print(f"Tensor x shape :   {x.shape}")
print(f"Tensor x dtype :   {x.dtype}")

Tensor x 
 [[1 2 4]
 [3 5 6]]
Tensor x shape :   (2, 3)
Tensor x dtype :   <dtype: 'int32'>


In [30]:
y = tf.convert_to_tensor(np.array([[1, 0, 1],[1, 1, 1]]))
y

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 0, 1],
       [1, 1, 1]])>

In [31]:
### Tensor operations

print(f"Transpose of x \n {x @ tf.transpose(x)}")

print(f"Concat x & y \n {tf.concat([x,y], axis=0)}")

print(f"Reduce sum {tf.reduce_sum([1,2,3])}")

Transpose of x 
 [[21 37]
 [37 70]]
Concat x & y 
 [[1 2 4]
 [3 5 6]
 [1 0 1]
 [1 1 1]]
Reduce sum 6


### Variables

Normal tf.Tensor objects are immutable. To store model weights (or other mutable state) in TensorFlow use a `tf.Variable`.

In [37]:
x_var = tf.Variable(x)
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [33]:
### Adding values to varible within the same ref
x_var.assign_add(y)
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[2, 2, 5],
       [4, 6, 7]])>

In [38]:
x_var.assign([[1, 2, 4],
       [3, 5, 6]])
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [39]:
#Most tensor operations work on variables as expected, although variables cannot be reshaped. 
## its creating tensor not varible

x_var_re = tf.reshape(x_var, [3,2])
x_var_re

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [4, 3],
       [5, 6]])>

### Automatic differentiation
Implementing automatic differentiation (autodiff), which uses calculus to compute gradients. Typically you'll use this to calculate the gradient of a model's error or loss with respect to its weights.

Gradient tapes
TensorFlow provides the `tf.GradientTape` API for automatic differentiation;
Once you've recorded some operations, use `GradientTape.gradient`(target, sources) to calculate the gradient of some target (often a loss) relative to some source (often the model's variables):

In [4]:
x = tf.Variable(2.0)

def fun(x):
    y = x**2+2*x+5
    
    return y

fun(x)

<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

In [ ]:
with tf.GradientTape() as tape:
    y = fun(x)

g_x = tape.gradient(y, x)  # g(x) = dy/dx  = 2x+2
g_x

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [6]:
## Gradient with two varibles

w = tf.Variable(tf.random.normal((3,2)), name='w')
b = tf.Variable(tf.zeros(2), name='b')

x=[[1., 2., 3.]]

In [7]:
with tf.GradientTape() as tape:
    y = x @ w +b
    loss = tf.reduce_mean(y**2)

In [8]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

In [11]:
w, b

(<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.49473074, -2.23581   ],
        [-1.1306745 ,  0.03210247],
        [-1.1583433 , -1.6996672 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>)

In [10]:
dl_dw, dl_db

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ -5.241648,  -7.270607],
        [-10.483296, -14.541214],
        [-15.724945, -21.811821]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.241648, -7.270607], dtype=float32)>)